# Exploring Zero-Shot and Multi-Shot Prompting with FLAN-T5

## Introduction
In this notebook, we explore different prompting strategies—**zero-shot** and **multi-shot prompting**—to generate dialogue summaries using a **pre-trained Large Language Model (LLM), FLAN-T5** from Hugging Face.

### Objective
We aim to:
- Generate dialogue summaries using **zero-shot** and **multi-shot** prompting.
- Compare the effectiveness of both approaches.
- Evaluate performance using **ROUGE scores**.

### Dataset
We will use the **DialogSum** dataset from Hugging Face, which contains over 10,000 dialogues with manually labeled summaries.

### Model
For this task, we will use **FLAN-T5**, a fine-tuned variant of T5 optimized for instruction-following and reasoning tasks.

Let's get started by setting up the environment and loading the necessary dependencies.


In [6]:

!pip install --upgrade pip
!pip install transformers
!pip install datasets --quiet
!pip install torchdata
!pip install torch
!pip install streamlit
!pip install openai
!pip install langchain
!pip install unstructured
!pip install sentence-transformers
!pip install chromadb
!pip install evaluate==0.4.0
!pip install rouge_score==0.1.2
!pip install loralib==0.1.1
!pip install peft==0.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1


In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import torch
import evaluate
import time
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import (AutoModelForSeq2SeqLM, AutoModelForCausalLM, 
                          AutoTokenizer, GenerationConfig, TrainingArguments, Trainer)
from transformers import AutoTokenizer
from transformers import GenerationConfig


Let's upload some simple dialogues from the DialogSum Hugging Face dataset. This dataset contains 10.000+ dialogues with the corresponding manually labeled summaries and topics.

In [9]:
DEVICE="mps"
torch_device = torch.device(DEVICE)

In [10]:
huggingface_dataset_name = 'knkarthick/dialogsum'
dataset = load_dataset(huggingface_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [11]:
example_indices = [40, 200]

In [12]:
dash_line = "-"*100

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print('Input Dialogue: ')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('Baseline Human Summary: ')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()
    

----------------------------------------------------------------------------------------------------
Example  1
----------------------------------------------------------------------------------------------------
Input Dialogue: 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
----------------------------------------------------------------------------------------------------
Baseline Human Summary: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
----------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------

## **FLAN-T5 Model**

![FLAN-T5 Architecture](images/flan2_architecture.jpg)

In [13]:
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

## Training

![FLAN-T5 Tasks](images/flan_t5_tasks.png)

#### These models are based on pretrained T5 (Raffel et al., 2020) and fine-tuned with instructions for better zero-shot and few-shot performance. There is one fine-tuned Flan model per T5 model size.

## Inference

In [20]:
sentence = "What time is it, Tom?"

In [21]:
sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(sentence_encoded["input_ids"][0], skip_special_tokens=True)

print(f'ENCODED SENTENCE:\n {sentence_encoded["input_ids"][0]}')
print(f'DECODED SENTENCE: {sentence_decoded}')

ENCODED SENTENCE:
 tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])
DECODED SENTENCE: What time is it, Tom?


## Zero-Shot Prompting for Dialogue Summarization

In this section, we use **zero-shot prompting**, where the model generates summaries **without any prior examples or fine-tuning**. We provide the raw dialogue as input and let **FLAN-T5** generate a summary based purely on its pretrained knowledge.


In [22]:
def summarize_dialogues(example_indices, dataset, prompt = "%s"):
    for i, index in enumerate(example_indices):
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
    
        input = prompt % (dialogue)
        
        inputs = tokenizer(input, return_tensors='pt')
        pred = model.generate(inputs["input_ids"], max_new_tokens=50)[0]
        output = tokenizer.decode(pred, skip_special_tokens=True)
    
        print(dash_line)
        print(f'Example {i+1}')
        print(dash_line)
        print(f'Input Prompt: \n {dialogue}')
        print(dash_line)
        print(f'Baseline Human Summary: \n {summary}')
        print(dash_line)
        print(f'Model Generation: \n{output}\n')

In [23]:
summarize_dialogues(example_indices, dataset)

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Prompt: 
 #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation: 
Person1: It's ten to nine.

--------------------------------------------------------

### Zero Shot Inference with an Instruction Prompt

In [24]:
prompt = f'Summarize the following conversation. \n%s\nSummary:'
print(prompt)

Summarize the following conversation. 
%s
Summary:


In [25]:
summarize_dialogues(example_indices, dataset, prompt)

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Prompt: 
 #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation: 
The train is about to leave.

------------------------------------------------------

In [26]:
prompt = f'Dialogue: \n%s\n\nWhat Happened?'
print(prompt)

Dialogue: 
%s

What Happened?


In [27]:
summarize_dialogues(example_indices, dataset, prompt)

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Prompt: 
 #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation: 
Tom is late.

----------------------------------------------------------------------

## One-Shot Inference

In **one-shot prompting**, we provide the model with **a single example** (dialogue + summary) before asking it to generate a summary for a new dialogue.

This method helps the model better understand the expected output format and improves response quality compared to **zero-shot prompting**.



In [28]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        
        prompt += f"""Dialogue:\n{dialogue}\n\nWhat was going on?\n{summary}\n\n\n"""
        dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f'Dialogue:\n{dialogue}\n\nWhat was going on?'
    return prompt

In [29]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)

Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


Dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a

In [30]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(inputs["input_ids"], max_new_tokens=50)[0], skip_special_tokens=True
)

print(dash_line)
print(f'Baseline Human Summary: \n{summary}\n')
print(dash_line)
print(f'Model Generation - One Shot:\n{output}')

---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
Model Generation - One Shot:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


## Few-Shot Inference

In **few-shot prompting**, we provide the model with **multiple examples** (dialogue-summary pairs) before asking it to generate a summary for a new dialogue.


In [31]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)

Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


Dialogue:
#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. May,

In [32]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(inputs["input_ids"], max_new_tokens=50)[0], skip_special_tokens=True
)

print(dash_line)
print(f'Baseline Human Summary: \n{summary}\n')
print(dash_line)
print(f'Model Generation - Few Shot:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (818 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
Model Generation - Few Shot:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.


## Summary: Comparing Zero-Shot, One-Shot, and Few-Shot Dialogue Summarization

### **Observations:**
- **Zero-Shot Output:** The model struggles to fully understand the task without examples, often producing incomplete or generic summaries.
- **One-Shot Output:** Adding a single example improves structure, but inconsistencies still appear.
- **Few-Shot Output:** Performance improves further, generating more structured summaries, but results may still lack **task-specific accuracy**.

### **Performance Improvement & Limitations**
- **Few-shot prompting enhances response quality** but still has **limitations:**
  - Requires longer context, which may **exceed token limits**.
  - Can still produce **inconsistent outputs**.
  - May not **fully adapt to domain-specific data**.

### **When to Choose Fine-Tuning Over Prompting?**
- **Fine-tuning is necessary when:**
  - The task requires **domain-specific language** not well-covered in pretraining.
  - **Prompting isn't reliable** across multiple cases.
  - You need **consistent behavior** and performance across a dataset.
  - **Inference cost needs to be optimized**, as fine-tuned models require **fewer tokens per request** compared to multi-shot prompting.

### **Next Steps**
To fully explore fine-tuning, refer to the following notebooks:
- **[Placeholder: Full Fine-Tuning Notebook]**
- **[Placeholder: PEFT/LoRA Fine-Tuning Notebook]**

These will provide an end-to-end implementation of full fine-tuning and **parameter-efficient fine-tuning (PEFT)** methods like **LoRA** to further enhance model performance.
